In [3]:
import re
from collections import defaultdict

In [4]:
input_file = "22_input.txt"
#input_file = "22_sample.txt"

In [14]:
class Node:
    def __init__(self, y, x):
        self.x = x
        self.y = y
        self.neighbours = {d: None for d in self.directions}

    directions = ["right", "down", "left", "up"]

class Board:
    def __init__(self, map_instructions):
        self.nodes = defaultdict(dict)
        self.coords = defaultdict(list)
        self.instructions = re.split(r"([LR])", map_instructions[-1])
        mapstrings = map_instructions[:-2]
        n_col = 0
        for j, r in enumerate(mapstrings):
            n_col = max(n_col, len(r))
            for k, c in enumerate(r):
                if c == '.':
                    self.nodes[j][k] = Node(j, k)
                    self.coords[j].append(k)
                    if k in self.coords[j - 1]:
                        self.nodes[j][k].neighbours["up"] = self.nodes[j - 1][k]
                        self.nodes[j-1][k].neighbours["down"] = self.nodes[j][k]
                    if k - 1 in self.coords[j]:
                        self.nodes[j][k].neighbours["left"] = self.nodes[j][k - 1]
                        self.nodes[j][k - 1].neighbours["right"] = self.nodes[j][k]
        # add periodicity (horizontal)
        for j, r in enumerate(mapstrings):
            row = r.strip()
            if row[0] == '.' and row[-1] == '.':
                start = self.nodes[j][self.coords[j][0]]
                end = self.nodes[j][self.coords[j][-1]]
                start.neighbours["left"] = end
                end.neighbours["right"] = start
        # add periodicity (vertical):
        for k in range(n_col):
            first_i = 0
            while len(mapstrings[first_i]) <= k or mapstrings[first_i][k] == ' ':
                first_i += 1
            last_i = len(mapstrings) - 1
            while len(mapstrings[last_i]) <= k or mapstrings[last_i][k] == ' ':
                last_i -= 1
            if mapstrings[first_i][k] == '.' and mapstrings[last_i][k] == '.':
                start = self.nodes[first_i][k]
                end = self.nodes[last_i][k]
                start.neighbours["up"] = end
                end.neighbours["down"] = start
        
        # clean dicts
        self.nodes = {k: v for k, v in self.nodes.items() if v}
        self.coords = {k: v for k, v in self.coords.items() if v}
        
    directions = ["right", "down", "left", "up"]
    def path(self):
        location = self.nodes[0][self.coords[0][0]] #first node
        facing = 0
        for i in self.instructions:
            if i == "L":
                facing = (facing - 1) % 4
            elif i == "R":
                facing = (facing + 1) % 4
            else:
                direction = self.directions[facing]
                for _ in range(int(i)):
                    if location.neighbours[direction] is None:
                        break
                    else:
                        location = location.neighbours[direction]
        password = 1000 * (location.y + 1) + 4 * (location.x + 1) + facing
        return password, location.y, location.x, facing

In [11]:
with open(input_file) as f:
    inputstrings = [line.rstrip('\n') for line in f] 

In [12]:
myboard = Board(inputstrings)

In [13]:
myboard.path()

(155060, 154, 14, 0)